## BCycle Data 
Notebook to predict the number of bikes leaving from each station per hour

## Getting the data from Blob Storage
Our data is stored inside of Azure Blob Storage. Following is the code to get it to your notebook.

In [45]:
# If you are running this code on your own computer, uncomment the following line.
# !pip install azure

In [46]:
from azure.storage.blob import BlockBlobService
import os

In [47]:
# You should only run this code once.
!mkdir bcycle

mkdir: cannot create directory ‘bcycle’: File exists


In [48]:
local_path=os.getcwd() + "/bcycle" # Where the data will be stored.
blob_account_name = "mldsdatahack2019diag" # Don't edit!
blob_account_key = "JsauBssnY92CeD3MgI2SWhkQ16JioJCRWVW8NzKtcWckI+DaNNbCmpmMAVq27GD91mhgH+oHPx+QbIKUCow5gA=="  # Don't edit!

mycontainer = "datahackdata2019"       # Don't edit!

In [49]:
import azure
from azure.storage.blob import BlockBlobService

# Create a client to get data
blob_service = BlockBlobService(account_name=blob_account_name, account_key=blob_account_key)
# Get the data
blob_service.get_blob_to_path("datahackdata2019", "BCycleAustin.csv", os.path.join(local_path, "BCycleAustin.csv"))

## Load Data

In [50]:
# Yo should see BCycleAustin.csv
!ls bcycle

BCycleAustin.csv


In [51]:
import pandas as pd
import matplotlib.pyplot as plt

In [93]:
# Read in data
df = pd.read_csv('bcycle/BCycleAustin.csv')

In [94]:
# See the first few rows.
df.head()

,Trip ID,Membership Type,Bicycle ID,Checkout Kiosk ID,Checkout Kiosk,Return Kiosk ID,Return Kiosk,Trip Duration Minutes,Checkout DateTime,Checkout Latitude,Checkout Longitude,Return Latitude,Return Longitude
0,9900285854,Annual (San Antonio B-cycle),207.0,2537.0,West & 6th St.,2707.0,Rainey St @ Cummings,76,2014-10-26 13:12:00,30.27041,-97.75046,30.25579,-97.73982
1,9900288020,24-Hour Kiosk (Austin B-cycle),190.0,2537.0,West & 6th St.,2707.0,Rainey St @ Cummings,12,2014-10-30 19:12:00,30.27041,-97.75046,30.25579,-97.73982
2,9900289011,Local365,190.0,2537.0,West & 6th St.,2707.0,Rainey St @ Cummings,9,2015-10-01 19:12:27,30.27041,-97.75046,30.25579,-97.73982
3,9900292552,Walk Up,354.0,2537.0,West & 6th St.,2707.0,Rainey St @ Cummings,14,2015-10-04 13:12:41,30.27041,-97.75046,30.25579,-97.73982
4,9900293709,Local365,20.0,2537.0,West & 6th St.,2707.0,Rainey St @ Cummings,11,2015-10-04 13:12:56,30.27041,-97.75046,30.25579,-97.73982


In [95]:
df.info(())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1027711 entries, 0 to 1027710
Data columns (total 13 columns):
Trip ID                  1027711 non-null int64
Membership Type          1027711 non-null object
Bicycle ID               1027711 non-null float64
Checkout Kiosk ID        1027711 non-null float64
Checkout Kiosk           1027711 non-null object
Return Kiosk ID          1027711 non-null float64
Return Kiosk             1027711 non-null object
Trip Duration Minutes    1027711 non-null int64
Checkout DateTime        1027711 non-null object
Checkout Latitude        1027711 non-null float64
Checkout Longitude       1027711 non-null float64
Return Latitude          1027711 non-null float64
Return Longitude         1027711 non-null float64
dtypes: float64(7), int64(2), object(4)
memory usage: 101.9+ MB


##  Get the hour when a bike left

In [96]:
# Convert the checkout datetime from string to an actual date-time object.
df['Checkout DateTime'] = pd.to_datetime(df['Checkout DateTime'])

In [97]:
# Checkout datetime is now a datetime
df.head()

,Trip ID,Membership Type,Bicycle ID,Checkout Kiosk ID,Checkout Kiosk,Return Kiosk ID,Return Kiosk,Trip Duration Minutes,Checkout DateTime,Checkout Latitude,Checkout Longitude,Return Latitude,Return Longitude
0,9900285854,Annual (San Antonio B-cycle),207.0,2537.0,West & 6th St.,2707.0,Rainey St @ Cummings,76,2014-10-26 13:12:00,30.27041,-97.75046,30.25579,-97.73982
1,9900288020,24-Hour Kiosk (Austin B-cycle),190.0,2537.0,West & 6th St.,2707.0,Rainey St @ Cummings,12,2014-10-30 19:12:00,30.27041,-97.75046,30.25579,-97.73982
2,9900289011,Local365,190.0,2537.0,West & 6th St.,2707.0,Rainey St @ Cummings,9,2015-10-01 19:12:27,30.27041,-97.75046,30.25579,-97.73982
3,9900292552,Walk Up,354.0,2537.0,West & 6th St.,2707.0,Rainey St @ Cummings,14,2015-10-04 13:12:41,30.27041,-97.75046,30.25579,-97.73982
4,9900293709,Local365,20.0,2537.0,West & 6th St.,2707.0,Rainey St @ Cummings,11,2015-10-04 13:12:56,30.27041,-97.75046,30.25579,-97.73982


In [138]:
# Get the checkout year, month, day, hour
df['Checkout Hour'] = df['Checkout DateTime'].apply(lambda x : x.hour)
df['Checkout Day'] = df['Checkout DateTime'].apply(lambda x: x.day)
df['Checkout Month'] = df['Checkout DateTime'].apply(lambda x: x.month)
df['Checkout Year'] = df['Checkout DateTime'].apply(lambda x: x.year)


In [139]:
df.head()

,Trip ID,Membership Type,Bicycle ID,Checkout Kiosk ID,Checkout Kiosk,Return Kiosk ID,Return Kiosk,Trip Duration Minutes,Checkout DateTime,Checkout Latitude,Checkout Longitude,Return Latitude,Return Longitude,Checkout Hour,Checkout Day,Checkout Year,Checkout Month
0,9900285854,Annual (San Antonio B-cycle),207.0,2537.0,West & 6th St.,2707.0,Rainey St @ Cummings,76,2014-10-26 13:12:00,30.27041,-97.75046,30.25579,-97.73982,13,26,2014,10
1,9900288020,24-Hour Kiosk (Austin B-cycle),190.0,2537.0,West & 6th St.,2707.0,Rainey St @ Cummings,12,2014-10-30 19:12:00,30.27041,-97.75046,30.25579,-97.73982,19,30,2014,10
2,9900289011,Local365,190.0,2537.0,West & 6th St.,2707.0,Rainey St @ Cummings,9,2015-10-01 19:12:27,30.27041,-97.75046,30.25579,-97.73982,19,1,2015,10
3,9900292552,Walk Up,354.0,2537.0,West & 6th St.,2707.0,Rainey St @ Cummings,14,2015-10-04 13:12:41,30.27041,-97.75046,30.25579,-97.73982,13,4,2015,10
4,9900293709,Local365,20.0,2537.0,West & 6th St.,2707.0,Rainey St @ Cummings,11,2015-10-04 13:12:56,30.27041,-97.75046,30.25579,-97.73982,13,4,2015,10


In [242]:
bikes_left = df.groupby(['Checkout Kiosk ID','Checkout Year','Checkout Month','Checkout Day','Checkout Hour'],sort=False).size().reset_index(name='Count')
bikes_left = bikes_left.drop(columns=['Checkout Year','Checkout Month','Checkout Day'])
bikes_left = bikes_left.dropna()

bikes_arriving = df.groupby(['Return Kiosk ID','Checkout Year','Checkout Month','Checkout Day','Checkout Hour'],sort=False).size().reset_index(name='Count')
bikes_arriving = bikes_arriving.drop(columns=['Checkout Year','Checkout Month','Checkout Day'])
bikes_left = bikes_left.dropna()

In [243]:
bikes_left.head()

,Checkout Kiosk ID,Checkout Hour,Count
0,2537.0,13,3
1,2537.0,19,3
2,2537.0,19,7
3,2537.0,13,3
4,2537.0,17,1


In [244]:
bikes_left.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 420261 entries, 0 to 420260
Data columns (total 3 columns):
Checkout Kiosk ID    420261 non-null float64
Checkout Hour        420261 non-null int64
Count                420261 non-null int64
dtypes: float64(1), int64(2)
memory usage: 12.8 MB


In [245]:
bikes_left["Checkout Kiosk ID"] = bikes_left["Checkout Kiosk ID"].astype('category')
bikes_arriving["Return Kiosk ID"] = bikes_arriving["Return Kiosk ID"].astype('category')
bikes_left.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 420261 entries, 0 to 420260
Data columns (total 3 columns):
Checkout Kiosk ID    420261 non-null category
Checkout Hour        420261 non-null int64
Count                420261 non-null int64
dtypes: category(1), int64(2)
memory usage: 10.0 MB


In [246]:
# Convert categorical variable into dummy/indicator variables
bikes_left = pd.get_dummies(bikes_left)
bikes_arriving = pd.get_dummies(bikes_arriving)

In [247]:
bikes_left.head()

,Checkout Hour,Count,Checkout Kiosk ID_1001.0,Checkout Kiosk ID_1002.0,Checkout Kiosk ID_1003.0,Checkout Kiosk ID_1005.0,Checkout Kiosk ID_1006.0,Checkout Kiosk ID_1007.0,Checkout Kiosk ID_1008.0,Checkout Kiosk ID_2494.0,...,Checkout Kiosk ID_4051.0,Checkout Kiosk ID_4052.0,Checkout Kiosk ID_4054.0,Checkout Kiosk ID_4055.0,Checkout Kiosk ID_4057.0,Checkout Kiosk ID_4058.0,Checkout Kiosk ID_4059.0,Checkout Kiosk ID_4060.0,Checkout Kiosk ID_4061.0,Checkout Kiosk ID_4062.0
0,13,3,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,19,3,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,19,7,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,13,3,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,17,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
